## HW-2 Part-2 - consists of Q4 of hw2

In [1]:
'''
Importing all libraries
'''
from copy import deepcopy
from numpy import argmax
import contractions
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import nltk
import torch
import gensim
import warnings
from sklearn.metrics import accuracy_score
from numpy import vstack
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import gensim.downloader as api
from sklearn.svm import LinearSVC as SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix as cm
from sklearn.linear_model import Perceptron
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('wordnet')
warnings.filterwarnings('ignore')
CUDA_LAUNCH_BLOCKING = 1

/home/darkghost/anaconda3/envs/ml/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/darkghost/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Transformation class for pre-processing

In [2]:
class DataTranformation(object):

    def __init__(self, filename, preprocess):
        self.filename = filename
        self.random_state = 10
        self.n = 50000
        self.preprocess = preprocess
        print("Preproces: " + str(preprocess))

    def read_file(self, error_bad_lines=False, warn_bad_lines=False, sep="\t"):
        df = pd.read_csv(self.filename, sep=sep,
                         error_bad_lines=error_bad_lines, warn_bad_lines=warn_bad_lines)
        df = df.dropna()
        return df

    def df_formation(self, row1='review_body', row2='star_rating', ):
        df = self.read_file()
        df = df[[row1, row2]]
        df = df.dropna()

        dataset = pd.concat([df[df['star_rating'] == 1].sample(n=50000, random_state=10),
                             df[df['star_rating'] == 2].sample(
                                 n=50000, random_state=10),
                             df[df['star_rating'] == 3].sample(
                                 n=50000, random_state=10),
                             df[df['star_rating'] == 4].sample(
                                 n=50000, random_state=10),
                             df[df['star_rating'] == 5].sample(n=50000, random_state=10)])

        dataset = dataset.reset_index(drop=True)

        return dataset

    def label(self, rows):
        if rows.star_rating > 3:
            return 1
        elif rows.star_rating < 3:
            return 2
        else:
            return 3

    def apply_label(self):
        dataset = self.df_formation()
        dataset['label'] = dataset.apply(lambda row: self.label(row), axis=1)

        return dataset

    def remove_html_and_url(self, s):
        s = re.sub(
            r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', '', s, flags=re.MULTILINE)
        soup = BeautifulSoup(s, 'html.parser')
        s = soup.get_text()
        return s

    def tokenize(self, s):
        text_tokens = word_tokenize(s)
        return text_tokens

    def without_preprocess(self):
        dataset = self.apply_label()
        dataset.review_body = dataset.review_body.apply(self.tokenize)
        return dataset

    def with_preprocess(self):
        dataset = self.apply_label()
        dataset.review_body = dataset.review_body.str.lower()

        dataset.review_body = dataset.review_body.apply(
            lambda s: self.remove_html_and_url(s))
        dataset.review_body = dataset.review_body.apply(
            lambda s: re.sub("[^a-zA-Z']+", " ", s))
        dataset.review_body = dataset.review_body.apply(
            lambda s: re.sub(' +', ' ', s))

        dataset.review_body = dataset.review_body.apply(self.tokenize)

        dataset.dropna()
        return dataset

    def train_test_split(self):

        if self.preprocess:
            dataset = self.with_preprocess()
        else:
            dataset = self.without_preprocess()

        train = dataset.sample(frac=0.8, random_state=200)
        test = dataset.drop(train.index)
        train = train.reset_index(drop=True)
        test = test.reset_index(drop=True)

        return train, test


### Vectorization Class for feature Extraction 

In [3]:
class Vectorization(object):

    def __init__(self, model, dataset, model_type="model", classification="binary", mode="mean", pad=False):
        self.model = model
        self.dataset = dataset
        self.model_type = model_type  
        self.classification = classification  
        if self.model_type == "pretrained":
            self.vocab = self.model
        if self.model_type == "model":
            self.vocab = self.model.wv

        self.mode = mode
        self.pad = pad

    def get_mean_vector(self, data_review_body, data_label):

        if self.classification == "binary":
            if data_label != 3:
                if self.model_type == "model":
                    words = [
                        word for word in data_review_body if word in self.vocab.index_to_key]
                    if len(words) >= 1:
                        rev = []
                        for word in words:
                            rev.append(np.array(self.vocab[word]))

                        if type(data_label) is not int:
                            print("Found")
                        return rev, data_label
                else:
                    words = [
                        word for word in data_review_body if word in self.vocab]
                    if len(words) >= 1:
                        rev = []
                        for word in words:
                            rev.append(np.array(self.vocab[word]))

                        if type(data_label) is not int:
                            print("Found")
                        return rev, data_label

        else:
            if self.model_type == "mode":
                words = [
                    word for word in data_review_body if word in self.vocab.index_to_key]
                if len(words) >= 1:
                    rev = []
                    for word in words:
                        rev.append(np.array(self.vocab[word]))
                    return rev, data_label
            else:
                words = [word for word in data_review_body if word in self.vocab]
                if len(words) >= 1:
                    rev = []
                    for word in words:
                        rev.append(np.array(self.vocab[word]))
                    return rev, data_label

    def feature_extraction(self):
        feature = []
        y_label = []
        # print(self.vocab.index_to_key)
        for data_review_body, data_label in zip(self.dataset.review_body, self.dataset.label):
            try:
                x, y = self.get_mean_vector(data_review_body, data_label)
                if self.pad:
                    if len(x) >= 50:
                        feature.append(x[:50])
                        y_label.append(y)
                    else:
                        feature.append(x)
                        y_label.append(y)
                else:
                    if self.mode == "vec":
                        if len(x) >= 10:
                            feature.append(x[:10])
                            y_label.append(y)
                    else:
                        feature.append(np.mean(x, axis=0))
                        y_label.append(y)
            except:
                pass
        print("Vectorization Completed")
        return feature, y_label

    def pad_review(self, review, seq_len):

        features = np.zeros((seq_len, 300), dtype=float)
        features[-len(review):] = np.array(review)[:seq_len]

        return features

    def join_words(self, x):
        y = ""
        for ele in x:
            y = ' '.join(ele)
        return y

In [4]:
class Sentence(object):
    def __init__(self, dataset):
        self.dataset = dataset

    def __iter__(self):
        for row in self.dataset:
            yield row

### Multi-layer Perceptron using average word2Vec similar to "Simple Models"

In [5]:
class MLP(nn.Module):
    def __init__(self, classification="binary", vocab_size=300):
        super(MLP, self).__init__()
        hidden_1 = 50
        hidden_2 = 10
        if classification == "binary":
            self.fc3 = nn.Linear(hidden_2, 3)
        else:
            # For multi-classification
            self.fc3 = nn.Linear(hidden_2, 4)
        self.fc1 = nn.Linear(vocab_size, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.sig = nn.Sigmoid()
        self.soft = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.view(-1, x.shape[1])
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

### Multi-layer Perceptron using first 10 Word2Vec features as input features

In [6]:
class MLP_vec(nn.Module):
    def __init__(self, classification="binary", vocab_size=300):
        super(MLP_vec, self).__init__()
        hidden_1 = 50
        hidden_2 = 10
        if classification == "binary":
            self.fc3 = nn.Linear(hidden_2, 3)
        else:
            # For multi-classification
            self.fc3 = nn.Linear(hidden_2, 4)
        self.prod = 10
        self.fc1 = nn.Linear(vocab_size * self.prod, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.sig = nn.Sigmoid()
        self.soft = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.view(-1, x.shape[1] * x.shape[2])
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

#### Data Loaders for Train Data and Test Data, which supplies reviews one by one from the batches to the model.

In [7]:
class trainData(Dataset):

    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__(self):
        return len(self.X_data)


class testData(Dataset):

    def __init__(self, X_data, Y_data):
        self.X_data = X_data
        self.Y_data = Y_data

    def __getitem__(self, index):
        return self.X_data[index], self.Y_data[index]

    def __len__(self):
        return len(self.X_data)

In [8]:
filename = "./amazon_reviews_us_Kitchen_v1_00.tsv"
dt = DataTranformation(filename, True)

train, test = dt.train_test_split()

sentences = Sentence(train['review_body'])

pretrained_model = api.load('word2vec-google-news-300')
model = gensim.models.Word2Vec(
    sentences, vector_size=300, min_count=10, window=11, seed=200)

Preproces: True


Mean feature extraction

In [9]:
vec_train = Vectorization(model=model, dataset=train)
vec_test = Vectorization(model, test)

X_train_model, Y_train_model = vec_train.feature_extraction()
X_test_model, Y_test_model = vec_test.feature_extraction()

Vectorization Completed


MEAN MULTI-CLASS FEATURES EXTRACTION


In [10]:
vec_multi_train = Vectorization(model, train, classification="multi-class")
vec_multi_test = Vectorization(model, test, classification="multi-class")

X_train_multi, Y_train_multi = vec_multi_train.feature_extraction()
X_test_multi, Y_test_multi = vec_multi_test.feature_extraction()

Vectorization Completed


TEN FEATURES IN A SINGLE ROW FEATURE EXTRACTION

In [11]:
vec_mode_train = Vectorization(model, train, classification="binary", mode="vec")
vec_mode_test = Vectorization(model, test, classification="binary", mode="vec")

X_train_mode, Y_train_mode = vec_mode_train.feature_extraction()
X_test_mode, Y_test_mode = vec_mode_test.feature_extraction()

Vectorization Completed


TEN FEATURES IN A SINGLE ROW MULTI-CLASS FEATURES EXTRACTION

In [12]:
vec_mode_train_multi = Vectorization(model, train, classification="multi-class", mode="vec")
vec_mode_test_multi = Vectorization(model, test, classification="multi-class", mode="vec")

X_train_mode_multi, Y_train_mode_multi = vec_mode_train_multi.feature_extraction()
X_test_mode_multi, Y_test_mode_multi = vec_mode_test_multi.feature_extraction()


Vectorization Completed


PRETRAINED MODEL FEATURES EXTRACTION

In [13]:
vec2_train = Vectorization(model=pretrained_model, dataset=train, model_type="pretrained")
vec2_test = Vectorization(model=pretrained_model, dataset=test, model_type="pretrained")

X_train_pre, Y_train_pre = vec2_train.feature_extraction()
X_test_pre, Y_test_pre = vec2_test.feature_extraction()

Vectorization Completed


PRETRAINED MODEL MULTI-CLASS FEATURES EXTRACTION

In [14]:
vec2_multi_train = Vectorization(model=pretrained_model, dataset=train, classification="multi-class", model_type="pretrained")
vec2_multi_test = Vectorization(model=pretrained_model, dataset=test, classification="multi-class", model_type="pretrained")

X_train_multi_pre, Y_train_multi_pre = vec2_multi_train.feature_extraction()
X_test_multi_pre, Y_test_multi_pre = vec2_multi_test.feature_extraction()

Vectorization Completed


PRETRAINED MODE VEC BINARY FEATURES EXTRACTION

In [15]:
vec_mode_train_pre = Vectorization(model=pretrained_model, dataset=train, model_type="pretrained", mode="vec")
vec_mode_test_pre = Vectorization(model=pretrained_model, dataset=test, model_type="pretrained", mode="vec")

X_train_mode_pre, Y_train_mode_pre = vec_mode_train_pre.feature_extraction()
X_test_mode_pre, Y_test_mode_pre = vec_mode_test_pre.feature_extraction()


Vectorization Completed


PRETRAINED MDOE VEC MULTI-CLASS FEATURES EXTRACTION

In [16]:
vec_mode_train_multi_pre = Vectorization(model=pretrained_model, dataset=train, classification="multi-class", model_type="pretrained", mode="vec")
vec_mode_test_multi_pre = Vectorization(model=pretrained_model, dataset=test, classification="multi-class", model_type="pretrained", mode="vec")

X_train_mode_multi_pre, Y_train_mode_multi_pre = vec_mode_train_multi_pre.feature_extraction()
X_test_mode_multi_pre, Y_test_mode_multi_pre = vec_mode_test_multi_pre.feature_extraction()

Vectorization Completed


### TRAINING FUNCTION

In [17]:
def training(model, epoch, dataset_x, dataset_y, name="model"):

    device = torch.device('cuda')
    print(model)

    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(mlp_model.parameters(), lr=0.01)

    criterion = criterion.to(device)

    training_data = trainData(torch.FloatTensor(
        dataset_x), torch.LongTensor(dataset_y))
    train_loader = DataLoader(
        dataset=training_data, batch_size=16, shuffle=True)

    for epoch in range(epoch):

        train_loss = 0.0

        mlp_model.train()
        for input_data, label in train_loader:
            optimizer.zero_grad()
            output = mlp_model(input_data.to(device))
            # y_batch.unsqueeze(1) (label.unsqueeze(1)).to(device)
            loss = criterion(output, label.to(device))
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * input_data.size(1)

        train_loss = train_loss/len(train_loader.dataset)

    # print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))
    torch.save(mlp_model.state_dict(), name + str(epoch + 1) + '.pt')

### TEST FUNCTION

In [25]:
def testing(model, epoch, dataset_x, dataset_y, name="model"):

    device = torch.device('cpu')
    testing_data = testData(torch.FloatTensor(
        dataset_x), torch.LongTensor(dataset_y))
    test_loader = DataLoader(dataset=testing_data, batch_size=16)
    tmp = 0
    for i in range(1, epoch+1):
        model.load_state_dict(torch.load(name + str(i) + '.pt'))
        model = model.to(device)

        predictions, actual = list(), list()
        for test_data, test_label in test_loader:

            pred = mlp_model(test_data.to(device))
            pred = pred.detach().numpy()
            pred = argmax(pred, axis=1)
            target = test_label.numpy()
            target = target.reshape((len(target), 1))
            pred = pred.reshape((len(pred)), 1)
            pred = pred.round()
            predictions.append(pred)
            actual.append(target)

        predictions, actual = vstack(predictions), vstack(actual)
        acc = accuracy_score(actual, predictions)
        if acc > tmp:
            tmp = acc
    print('Accuracy: %.3f' % tmp)


In [21]:
device = torch.device('cuda')

In [31]:
""" BINARY-MEAN MLP """

mlp_model = MLP()  # binary classification
training(mlp_model, 10, X_train_model, Y_train_model, name="mlp_model")
testing(mlp_model, 10, X_test_model, Y_test_model, name="mlp_model")

Accuracy: 0.788


In [32]:

""" MULTI-CLASS MEAN MLP """

mlp_model = MLP(classification="multi-class")
training(mlp_model, 10, X_train_multi, Y_train_multi, name="mlp_model_multi")
testing(mlp_model, 10, X_test_multi, Y_test_multi, name="mlp_model_multi")


Accuracy: 0.629


In [33]:

""" BINARY-VEC MLP """

mlp_model = MLP_vec()
training(mlp_model, 10, X_train_mode, Y_train_mode, name="mlp_mode_vec")
testing(mlp_model, 10, X_test_mode, Y_test_mode, name="mlp_mode_vec")


Accuracy: 0.716


In [34]:

""" MULTI-VEC MLP """

mlp_model = MLP_vec(classification="multi-class")
training(mlp_model, 10, X_train_mode_multi, Y_train_mode_multi, name="mlp_mode_vec_multi")
testing(mlp_model, 10, X_test_mode_multi,
        Y_test_mode_multi, name="mlp_mode_vec_multi")


Accuracy: 0.568


In [35]:

""" BINARY-MEAN PRETRAINED MLP """
mlp_model = MLP()
training(mlp_model, 10, X_train_pre, Y_train_pre, name="mlp_model_pre")
testing(mlp_model, 10, X_test_pre, Y_test_pre, name="mlp_model_pre")


Accuracy: 0.838


In [36]:
""" MULTI-CLASS MEAN PRETRAINED MLP """
mlp_model = MLP(classification="multi-class")
training(mlp_model, 10, X_train_multi_pre, Y_train_multi_pre, name="mlp_mode_multi_pre")
testing(mlp_model, 10, X_test_multi_pre, Y_test_multi_pre, name="mlp_mode_multi_pre")


Accuracy: 0.679


In [37]:
""" BINARY VEC PRETRAINED MLP """

mlp_model = MLP_vec()
training(mlp_model, 10, X_train_mode_pre, Y_train_mode_pre, name="mlp_vec_pre")
testing(mlp_model, 10, X_test_mode_pre, Y_test_mode_pre, name="mlp_vec_pre")


Accuracy: 0.755


In [38]:
""" MULTI-CLASS VEC PRETRAINED MLP """

mlp_model = MLP_vec(classification="multi-class")
training(mlp_model, 10, X_train_mode_multi_pre, Y_train_mode_multi_pre, name="mlp_vec_multi_pre")
testing(mlp_model, 10, X_test_mode_multi_pre, Y_test_mode_multi_pre, name="mlp_vec_multi_pre")


Accuracy: 0.608


## Observation:

MLP trained using self-trained Word2Vec feature vectors produces better accuracy compared to pre-trained ones.